In [1]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display

import os

import pandas as pd
import sys

# Import package numpy for numeric computing
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
#For showing plots directly in the notebook run the command below
%matplotlib inline
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

#to split the data into test and train
from sklearn.model_selection import train_test_split

from random import randint

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

import random

from sklearn.metrics import accuracy_score

import pickle

In [2]:
url = "dbikes.cznzccwi0urk.us-east-1.rds.amazonaws.com"
user = "admin"
database= "dbikes"
port = "3306"
password = "Foryiuxing18!"

try:
    engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{database}", echo=True)
    print("successfully connected to database")
except:
    print("connection failed")

successfully connected to database


In [3]:
availability_data = pd.read_sql("select * from dbikes.availability_log", engine)

2023-04-21 10:06:39,206 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-21 10:06:39,209 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-21 10:06:39,406 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-21 10:06:39,408 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-21 10:06:39,508 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-21 10:06:39,509 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-04-21 10:06:39,809 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-21 10:06:39,812 INFO sqlalchemy.engine.Engine [generated in 0.00232s] ('dbikes', 'select * from dbikes.availability_log')
2023-04-21 10:06:40,022 INFO sqlalchemy.engine.Engine select * from dbikes.availability_log
2023-04-21 10:06:40,024 INFO sqlalchemy.engine.Engine [raw sql] ()


In [4]:
weather_data = pd.read_sql("select * from dbikes.weather", engine)

2023-04-21 10:06:43,548 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %s AND table_name = %s
2023-04-21 10:06:43,549 INFO sqlalchemy.engine.Engine [cached since 3.74s ago] ('dbikes', 'select * from dbikes.weather')
2023-04-21 10:06:43,748 INFO sqlalchemy.engine.Engine select * from dbikes.weather
2023-04-21 10:06:43,750 INFO sqlalchemy.engine.Engine [raw sql] ()


In [5]:

weather_data.shape

(828, 14)

In [6]:
availability_data.shape

(884017, 4)

In [7]:
#availability_data1=pd.read_csv("availability.csv")
#weather_data=pd.read_csv("weather.csv")
availability_data.shape

(884017, 4)

In [8]:
weather_data.tail(5)

,position_lng,position_lat,description,temperature,feels_like,temp_min,temp_max,pressure,humidity,visibility,wind_speed,dt,sunrise,sunset
823,-6,53,few clouds,8,4,7,9,1020,80,10000,9,1682045944,1682053964,1682105675
824,-6,53,few clouds,8,5,7,9,1020,80,10000,8,1682049558,1682053964,1682105675
825,-6,53,broken clouds,9,5,8,9,1019,80,10000,7,1682053204,1682053964,1682105675
826,-6,53,broken clouds,9,5,8,9,1018,81,10000,7,1682056803,1682053964,1682105675
827,-6,53,broken clouds,9,5,8,9,1017,84,10000,7,1682060404,1682053964,1682105675


In [9]:

availability_data.head(5)

,number,available_bikes,available_bike_stands,last_update
0,42,13,17,1678291428000
1,30,0,20,1678291358000
2,54,6,27,1678291268000
3,108,1,34,1678291419000
4,20,4,26,1678291259000


In [10]:
def date(x):
    formats = pd.to_datetime(x, unit='s')
    return formats

weather_data['dt'] = weather_data['dt'].apply(date)
weather_data['sunrise'] = weather_data['sunrise'].apply(date)
weather_data['sunset'] = weather_data['sunset'].apply(date)


In [11]:
#converting unixtime in availabilty from ms to date time
def date(x):
    formats = pd.to_datetime(x, unit='ms')
    return formats
availability_data['last_update'] = availability_data['last_update'].apply(date)

In [12]:

availability_data.tail(5)

,number,available_bikes,available_bike_stands,last_update
884012,39,0,19,2023-04-21 07:53:33
884013,83,19,21,2023-04-21 07:54:21
884014,92,29,9,2023-04-21 07:53:01
884015,21,17,12,2023-04-21 07:54:11
884016,88,20,10,2023-04-21 07:52:42


In [13]:
weather_data.tail(5)

,position_lng,position_lat,description,temperature,feels_like,temp_min,temp_max,pressure,humidity,visibility,wind_speed,dt,sunrise,sunset
823,-6,53,few clouds,8,4,7,9,1020,80,10000,9,2023-04-21 02:59:04,2023-04-21 05:12:44,2023-04-21 19:34:35
824,-6,53,few clouds,8,5,7,9,1020,80,10000,8,2023-04-21 03:59:18,2023-04-21 05:12:44,2023-04-21 19:34:35
825,-6,53,broken clouds,9,5,8,9,1019,80,10000,7,2023-04-21 05:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35
826,-6,53,broken clouds,9,5,8,9,1018,81,10000,7,2023-04-21 06:00:03,2023-04-21 05:12:44,2023-04-21 19:34:35
827,-6,53,broken clouds,9,5,8,9,1017,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35


In [14]:
weather_data.dtypes

position_lng             int64
position_lat             int64
description             object
temperature              int64
feels_like               int64
temp_min                 int64
temp_max                 int64
pressure                 int64
humidity                 int64
visibility               int64
wind_speed               int64
dt              datetime64[ns]
sunrise         datetime64[ns]
sunset          datetime64[ns]
dtype: object

In [15]:
availability_data.dtypes

number                            int64
available_bikes                   int64
available_bike_stands             int64
last_update              datetime64[ns]
dtype: object

In [16]:
availability_data.drop_duplicates(subset='last_update', keep='first', inplace=True)
weather_data.drop_duplicates(subset='dt', keep='first', inplace=True)



In [17]:
print('Weather shape: ', weather_data.shape)
print('Availability shape: ', availability_data.shape)

Weather shape:  (827, 14)
Availability shape:  (309548, 4)


In [18]:
print(availability_data.columns)
print(weather_data.columns)


Index(['number', 'available_bikes', 'available_bike_stands', 'last_update'], dtype='object')
Index(['position_lng', 'position_lat', 'description', 'temperature',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'visibility', 'wind_speed', 'dt', 'sunrise', 'sunset'],
      dtype='object')


#### Merging Dataframes

In [19]:
availability_data.rename(columns={'last_update':'dt'}, inplace=True)

In [20]:

availability_data['hour'] = availability_data['dt'].dt.hour
availability_data['minute'] = availability_data['dt'].dt.minute
availability_data['day'] = availability_data['dt'].dt.day
availability_data['month'] = availability_data['dt'].dt.month

availability_data['dayofweek'] = availability_data['dt'].dt.dayofweek
availability_data['dt_hour'] = availability_data['dt'].dt.hour

In [21]:

weather_data['hour'] = weather_data['dt'].dt.hour
weather_data['minute'] = weather_data['dt'].dt.minute
weather_data['day'] = weather_data['dt'].dt.day
weather_data['month'] = weather_data['dt'].dt.month

weather_data['dayofweek'] = weather_data['dt'].dt.dayofweek
weather_data['dt_hour'] = weather_data['dt'].dt.hour

In [22]:
#creating new pandas daraframe by merging 
newdf = pd.merge(availability_data, weather_data, on=['hour','hour', 'day', 'day'])

In [23]:
newdf.shape

(531827, 28)

In [24]:
newdf

,number,available_bikes,available_bike_stands,dt_x,hour,minute_x,day,month_x,dayofweek_x,dt_hour_x,...,humidity,visibility,wind_speed,dt_y,sunrise,sunset,minute_y,month_y,dayofweek_y,dt_hour_y
0,42,13,17,2023-03-08 16:03:48,16,3,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
1,30,0,20,2023-03-08 16:02:38,16,2,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
2,54,6,27,2023-03-08 16:01:08,16,1,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
3,108,1,34,2023-03-08 16:03:39,16,3,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
4,20,4,26,2023-03-08 16:00:59,16,0,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531822,103,0,40,2023-04-21 07:51:02,7,51,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7
531823,28,9,21,2023-04-21 07:50:11,7,50,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7
531824,39,0,19,2023-04-21 07:53:33,7,53,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7
531825,83,19,21,2023-04-21 07:54:21,7,54,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7


In [25]:
newdf


,number,available_bikes,available_bike_stands,dt_x,hour,minute_x,day,month_x,dayofweek_x,dt_hour_x,...,humidity,visibility,wind_speed,dt_y,sunrise,sunset,minute_y,month_y,dayofweek_y,dt_hour_y
0,42,13,17,2023-03-08 16:03:48,16,3,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
1,30,0,20,2023-03-08 16:02:38,16,2,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
2,54,6,27,2023-03-08 16:01:08,16,1,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
3,108,1,34,2023-03-08 16:03:39,16,3,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
4,20,4,26,2023-03-08 16:00:59,16,0,8,3,2,16,...,72,10000,5,2023-04-08 16:55:36,2023-04-08 05:42:36,2023-04-08 19:11:02,55,4,5,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531822,103,0,40,2023-04-21 07:51:02,7,51,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7
531823,28,9,21,2023-04-21 07:50:11,7,50,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7
531824,39,0,19,2023-04-21 07:53:33,7,53,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7
531825,83,19,21,2023-04-21 07:54:21,7,54,21,4,4,7,...,84,10000,7,2023-04-21 07:00:04,2023-04-21 05:12:44,2023-04-21 19:34:35,0,4,4,7


In [26]:
newdf.dtypes

number                            int64
available_bikes                   int64
available_bike_stands             int64
dt_x                     datetime64[ns]
hour                              int64
minute_x                          int64
day                               int64
month_x                           int64
dayofweek_x                       int64
dt_hour_x                         int64
position_lng                      int64
position_lat                      int64
description                      object
temperature                       int64
feels_like                        int64
temp_min                          int64
temp_max                          int64
pressure                          int64
humidity                          int64
visibility                        int64
wind_speed                        int64
dt_y                     datetime64[ns]
sunrise                  datetime64[ns]
sunset                   datetime64[ns]
minute_y                          int64


In [27]:
newdf.columns

Index(['number', 'available_bikes', 'available_bike_stands', 'dt_x', 'hour',
       'minute_x', 'day', 'month_x', 'dayofweek_x', 'dt_hour_x',
       'position_lng', 'position_lat', 'description', 'temperature',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'visibility', 'wind_speed', 'dt_y', 'sunrise', 'sunset', 'minute_y',
       'month_y', 'dayofweek_y', 'dt_hour_y'],
      dtype='object')

In [28]:
category_columns = newdf.select_dtypes(['object', 'category']).columns

In [29]:
continuous_columns = newdf.select_dtypes(['int', 'float']).columns

In [30]:
newdf[continuous_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
number,531827.0,57.660390,33.967882,1.0,28.0,57.0,87.0,507.0
available_bikes,531827.0,11.534093,9.468407,0.0,3.0,10.0,18.0,40.0
available_bike_stands,531827.0,20.076269,10.730763,0.0,12.0,20.0,28.0,40.0
hour,531827.0,11.930916,6.632507,0.0,6.0,12.0,17.0,23.0
minute_x,531827.0,29.720467,17.336480,0.0,15.0,30.0,45.0,59.0
day,531827.0,13.274115,4.677831,3.0,10.0,13.0,17.0,30.0
month_x,531827.0,3.997552,0.049456,2.0,4.0,4.0,4.0,4.0
dayofweek_x,531827.0,2.821948,1.921834,0.0,1.0,3.0,4.0,6.0
dt_hour_x,531827.0,11.930916,6.632507,0.0,6.0,12.0,17.0,23.0
position_lng,531827.0,-6.000000,0.000000,-6.0,-6.0,-6.0,-6.0,-6.0


In [31]:
newdf[category_columns].describe().T

,count,unique,top,freq
description,531827,17,broken clouds,264285


In [32]:
"""#Correlations for the numeric features for all stations
# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html


# Calculate correlation of all pairs of continuous features
corr = newdf[continuous_columns].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom colormap - blue and red
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,
            square=True, xticklabels=True, yticklabels=True,
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)
plt.yticks(rotation = 0)
plt.xticks(rotation = 45)"""

'#Correlations for the numeric features for all stations\n# Correlation matrix using code found on https://stanford.edu/~mwaskom/software/seaborn/examples/many_pairwise_correlations.html\n\n\n# Calculate correlation of all pairs of continuous features\ncorr = newdf[continuous_columns].corr()\n\n# Generate a mask for the upper triangle\nmask = np.zeros_like(corr, dtype=bool)\nmask[np.triu_indices_from(mask)] = True\n\n# Set up the matplotlib figure\nf, ax = plt.subplots(figsize=(11, 9))\n\n# Generate a custom colormap - blue and red\ncmap = sns.diverging_palette(220, 10, as_cmap=True)\n\n# Draw the heatmap with the mask and correct aspect ratio\nsns.heatmap(corr, annot=True, mask=mask, cmap=cmap, vmax=1, vmin=-1,\n            square=True, xticklabels=True, yticklabels=True,\n            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax)\nplt.yticks(rotation = 0)\nplt.xticks(rotation = 45)'

In [33]:
newdf['description'].unique()

array(['broken clouds', 'moderate rain', 'light rain',
       'light intensity drizzle', 'light intensity shower rain',
       'scattered clouds', 'few clouds', 'clear sky', 'snow', 'sleet',
       'ragged shower rain', 'light snow', 'heavy intensity rain', 'mist',
       'fog', 'overcast clouds', 'light intensity drizzle rain'],
      dtype=object)

In [34]:


# convert 'description' to one-hot encoded binary features
one_hot = pd.get_dummies(newdf['description'])

# join the one-hot encoded features to the original dataframe
newdf = newdf.join(one_hot)

# print the new dataframe
print(newdf)


        number  available_bikes  available_bike_stands                dt_x  \
0           42               13                     17 2023-03-08 16:03:48   
1           30                0                     20 2023-03-08 16:02:38   
2           54                6                     27 2023-03-08 16:01:08   
3          108                1                     34 2023-03-08 16:03:39   
4           20                4                     26 2023-03-08 16:00:59   
...        ...              ...                    ...                 ...   
531822     103                0                     40 2023-04-21 07:51:02   
531823      28                9                     21 2023-04-21 07:50:11   
531824      39                0                     19 2023-04-21 07:53:33   
531825      83               19                     21 2023-04-21 07:54:21   
531826      88               20                     10 2023-04-21 07:52:42   

        hour  minute_x  day  month_x  dayofweek_x  dt_hour_x  .

In [35]:
newdf

,number,available_bikes,available_bike_stands,dt_x,hour,minute_x,day,month_x,dayofweek_x,dt_hour_x,...,light intensity shower rain,light rain,light snow,mist,moderate rain,overcast clouds,ragged shower rain,scattered clouds,sleet,snow
0,42,13,17,2023-03-08 16:03:48,16,3,8,3,2,16,...,0,0,0,0,0,0,0,0,0,0
1,30,0,20,2023-03-08 16:02:38,16,2,8,3,2,16,...,0,0,0,0,0,0,0,0,0,0
2,54,6,27,2023-03-08 16:01:08,16,1,8,3,2,16,...,0,0,0,0,0,0,0,0,0,0
3,108,1,34,2023-03-08 16:03:39,16,3,8,3,2,16,...,0,0,0,0,0,0,0,0,0,0
4,20,4,26,2023-03-08 16:00:59,16,0,8,3,2,16,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531822,103,0,40,2023-04-21 07:51:02,7,51,21,4,4,7,...,0,0,0,0,0,0,0,0,0,0
531823,28,9,21,2023-04-21 07:50:11,7,50,21,4,4,7,...,0,0,0,0,0,0,0,0,0,0
531824,39,0,19,2023-04-21 07:53:33,7,53,21,4,4,7,...,0,0,0,0,0,0,0,0,0,0
531825,83,19,21,2023-04-21 07:54:21,7,54,21,4,4,7,...,0,0,0,0,0,0,0,0,0,0


In [36]:
monday = newdf[newdf.dayofweek_x == 0]
tuesday = newdf[newdf.dayofweek_x == 1]
wednesday = newdf[newdf.dayofweek_x == 2]
thursday = newdf[newdf.dayofweek_x == 3]
friday = newdf[newdf.dayofweek_x == 4]
saturday = newdf[newdf.dayofweek_x == 5]
sunday = newdf[newdf.dayofweek_x == 6]

In [37]:
monday.columns

Index(['number', 'available_bikes', 'available_bike_stands', 'dt_x', 'hour',
       'minute_x', 'day', 'month_x', 'dayofweek_x', 'dt_hour_x',
       'position_lng', 'position_lat', 'description', 'temperature',
       'feels_like', 'temp_min', 'temp_max', 'pressure', 'humidity',
       'visibility', 'wind_speed', 'dt_y', 'sunrise', 'sunset', 'minute_y',
       'month_y', 'dayofweek_y', 'dt_hour_y', 'broken clouds', 'clear sky',
       'few clouds', 'fog', 'heavy intensity rain', 'light intensity drizzle',
       'light intensity drizzle rain', 'light intensity shower rain',
       'light rain', 'light snow', 'mist', 'moderate rain', 'overcast clouds',
       'ragged shower rain', 'scattered clouds', 'sleet', 'snow'],
      dtype='object')

In [38]:
features= ['number','temperature', 'pressure', 'humidity', 'wind_speed','hour', 'dayofweek_x', 'month_x']

In [43]:
#data training 
from sklearn import metrics

X = newdf[features]
y = newdf.available_bikes

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor =RandomForestRegressor(n_estimators=15)

regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('available_bikes.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))

Test Set Predictions for Station: 
        Actual  Predicted
300872      27  29.225503
297840      30  29.108817
14424       15  13.594603
385611       9   9.411852
500757       0   0.000000
...        ...        ...
238918       3   4.072944
177922      11  11.579594
79948       24  23.084444
453259      23  23.000000
516919       2   2.000000

[159549 rows x 2 columns]

Mean Absolute Error: 0.8859927678383644
Mean Squared Error: 2.7769371216441896
Root Mean Squared Error: 1.6664144507427285
The accuracy score :  0.9689985392300694


In [45]:
#data training 
from sklearn import metrics

X = newdf[features]
y = newdf.available_bike_stands

X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

regressor =RandomForestRegressor(n_estimators=1)

regressor.fit(X_train, y_train)
    
# Write to a pickle file
with open('available_bike_stands.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
print('Test Set Predictions for Station: ')
print(df)
print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))

Test Set Predictions for Station: 
        Actual  Predicted
300872       3   0.606157
297840       0   0.784841
14424        1   1.664444
385611      11  10.146032
500757      20  20.000000
...        ...        ...
238918      29  26.915635
177922       9   8.481984
79948        1   1.758333
453259      17  17.000000
516919      38  38.000000

[159549 rows x 2 columns]

Mean Absolute Error: 0.8911447818542545
Mean Squared Error: 2.8445731620702563
Root Mean Squared Error: 1.6865862450732416
The accuracy score :  0.9752777454174378


In [43]:
def func():
    url = "https://api.openweathermap.org/data/2.5/forecast?q=Dublin,IE&units=metric&appid=925fb7d8a523499058239184ca7054d9"
    weather = requests.get(url)
    data = weather.json()
    
    predict_array = []
    json_dict = {}
    for hour in range(24):
        # Update the temperature, pressure, humidity and wind_speed values based on the API response
        temperature = data['list'][hour]['main']['temp']
        pressure = data['list'][hour]['main']['pressure']
        humidity = data['list'][hour]['main']['humidity']
        wind_speed = data['list'][hour]['wind']['speed']
        
        # Use the current date and time to get the hour, day of the week and month
        today = datetime.today()
        day_of_week, month = today.weekday(), today.month
        
        # Update the values in the predict_array
        predict_array.append([station_id, temperature, pressure, humidity, wind_speed, hour, day_of_week, month])
        results = bike_model.predict(predict_array).tolist()
    for index, bikes in enumerate(results):
        json_dict[index] = bikes
        
func()

NameError: name 'datetime' is not defined

In [49]:
url2 = "https://api.openweathermap.org/data/2.5/forecast?lat={53.35}&lon={6.2}&appid={925fb7d8a523499058239184ca7054d9}"
url = "https://api.openweathermap.org/data/2.5/forcast?q=Dublin,IE&units=metric&appid=925fb7d8a523499058239184ca7054d9"
weather = requests.get(url)
weather = weather.json()
weather

{'cod': '404', 'message': 'Internal error'}

In [57]:


df =pd.DataFrame({'number':[42], 'temperature': [-2], 'pressure': [997], 'humidity':[ 70], 'wind_speed': [6.6], 'hour': [3],
                      'broken clouds': [0], 'moderate rain': [0], 'light rain': [1], 'light intensity drizzle': [0], 'light intensity shower rain': [0],
                      'scattered clouds': [0], 'few clouds': [0], 'clear sky': [0], 'snow': [1], 'sleet': [0],
       'ragged shower rain': [0], 'light snow': [0], 'dayofweek_x': [7]}
                )

with open('bike_station.pkl', 'rb') as handle:
    regressor = pickle.load(handle) 
    
prediction = regressor.predict(df)




In [58]:
prediction[0]

26.412448412698414

In [113]:
@app.route('/availability/<int:station_id>')
def predict_bikes(station_id):
    # ['number','temperature', 'pressure', 'humidity', 'wind_speed','hour', 'dayofweek_x', 'month_x']
    from datetime import datetime
    today = datetime.today()
    day_of_week,month = today.weekday(),today.month

    predict_array = []
    json_dict = {}
    for hour in range(24):
        predict_array.append([station_id,0,99,99,99, hour, day_of_week,month])
    results = bike_model.predict(predict_array).tolist()
    for index,bikes in enumerate(results):
        json_dict[index] = bikes

    return json.dumps(json_dict)

@app.route('/standsavailability/<int:stand_id>')
def predict_stands(stand_id):
    # ['number','temperature', 'pressure', 'humidity', 'wind_speed','hour', 'dayofweek_x', 'month_x']
    from datetime import datetime
    today = datetime.today()
    dow,month = today.weekday(),today.month

    predict_array = []
    json_dict = {}
    for hour in range(24):
        predict_array.append([stand_id,0,99,99,99, hour, day_of_week, month])
    results = stands_model.predict(predict_array).tolist()
    for index,stands in enumerate(results):
        json_dict[index] = stands

    return json.dumps(json_dict)

(32, 34)

import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback
import glob
import os
from pprint import pprint
import json
import requests
import time
from IPython.display import display

url = "dbikes.cznzccwi0urk.us-east-1.rds.amazonaws.com"
user = "admin"
database= "dbikes"
port = "3306"
password = "Foryiuxing18!"

engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{database}", echo=True)


url = "https://api.openweathermap.org/data/2.5/weather?q=Dublin,IE&units=metric&appid=925fb7d8a523499058239184ca7054d9"
import datetime
def write_to_file2(text):
    now = datetime.datetime.now()
    with open(f"data/weather_{now}".replace(" ", "_"), "w") as f:
        f.write(text) 
        
def weather_to_db(text):
    weather = json.loads(text)
    print(type(weather),len(weather))
    print(text)
    now = datetime.datetime.now()
    vals = (
            weather.get("coord").get("lon"),
            weather.get("coord").get("lat"),
            weather.get("weather")[0].get("description"),
            weather.get("main").get("temp"),
            weather.get("main").get("feels_like"),
            weather.get("main").get("temp_min"),
            weather.get("main").get("temp_max"),
            weather.get("main").get("pressure"),
            weather.get("main").get("humidity"),
            weather.get("visibility"),
            weather.get("wind").get("speed"),
            weather.get("dt"),
            weather.get("sys").get("sunrise"),
            weather.get("sys").get("sunset"))
        
    
    
    print("\n\n\nVALS:{}\n\n\n".format(vals))
    engine.execute("insert into weather values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)
    
def weather_main():
    
    try:
        url = "https://api.openweathermap.org/data/2.5/weather?q=Dublin,IE&units=metric&appid=925fb7d8a523499058239184ca7054d9"

        weather = requests.get(url)
        write_to_file2(weather.text)
        weather_to_db(weather.text)
    except:
          print(traceback.format_exc())
          if engine is None:
                return

if __name__ == "__main__":
    weather_main() 

In [ ]:
omp